## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [1]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
#from keras.layers import GlobalAveragePooling2D

import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder


Using TensorFlow backend.


In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [3]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

C:\Anaconda3\envs\testAI\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [4]:

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32,(3,3),padding='same',input_shape=(32,32,3),activation='relu'))  #32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(32,(3,3),padding='same',activation='relu'))
classifier.add(BatchNormalization())
#classifier.add(MaxPooling2D(pool_size=(2,2)))

#flatten
classifier.add(Flatten())
#classifier.add(GlobalAveragePooling2D())

#FC
classifier.add(Dense(units=100, activation='relu')) #output_dim=100,activation=relu
#classifier.add(Dropout(rate = 0.25))

#輸出
classifier.add(Dense(output_dim=10,activation='softmax'))


#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)















Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


C:\Anaconda3\envs\testAI\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`




Epoch 1/5
50000/50000 [==============================] - 60s 1ms/step - loss: 1.2730 - acc: 0.5555
Epoch 2/5
50000/50000 [==============================] - 60s 1ms/step - loss: 0.8056 - acc: 0.7181
Epoch 3/5
50000/50000 [==============================] - 59s 1ms/step - loss: 0.5735 - acc: 0.7991
Epoch 4/5
50000/50000 [==============================] - 61s 1ms/step - loss: 0.3813 - acc: 0.8677
Epoch 5/5
50000/50000 [==============================] - 60s 1ms/step - loss: 0.2450 - acc: 0.9156


## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [5]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[4.77057695e-01, 4.18252330e-06, 8.42812732e-02, 4.02389973e-01,
        4.50884504e-03, 2.11138278e-03, 7.06974184e-04, 1.63764153e-02,
        1.24551952e-02, 1.08018285e-04]], dtype=float32)

day15作業參數與結果測試

(1)同12/13的原始程式碼 (batch_size=100,epochs=100)
Epoch 1/100
50000/50000 [==============================] - 60s 1ms/step - loss: 1.2490 - acc: 0.5679
Epoch 2/100
50000/50000 [==============================] - 59s 1ms/step - loss: 0.8047 - acc: 0.7174
Epoch 3/100
50000/50000 [==============================] - 58s 1ms/step - loss: 0.5768 - acc: 0.7987
Epoch 4/100
50000/50000 [==============================] - 59s 1ms/step - loss: 0.3911 - acc: 0.8645
Epoch 5/100
50000/50000 [==============================] - 58s 1ms/step - loss: 0.2515 - acc: 0.9139

(2) batch_size=150,epochs=5 (降低epochs是為了快速知道大概的趨勢)
Epoch 1/10
50000/50000 [==============================] - 59s 1ms/step - loss: 1.2846 - acc: 0.5530
Epoch 2/10
50000/50000 [==============================] - 58s 1ms/step - loss: 0.8200 - acc: 0.7116
Epoch 3/10
50000/50000 [==============================] - 61s 1ms/step - loss: 0.5768 - acc: 0.7993
Epoch 4/10
50000/50000 [==============================] - 59s 1ms/step - loss: 0.3811 - acc: 0.8685
Epoch 5/10
50000/50000 [==============================] - 58s 1ms/step - loss: 0.2455 - acc: 0.9156

(3) batch_size=200  ->增大兩倍沒有甚麼變化
Epoch 1/10
50000/50000 [==============================] - 60s 1ms/step - loss: 1.3132 - acc: 0.5472
Epoch 2/10
50000/50000 [==============================] - 60s 1ms/step - loss: 0.8059 - acc: 0.7187
Epoch 3/10
50000/50000 [==============================] - 58s 1ms/step - loss: 0.5432 - acc: 0.8103
Epoch 4/10
50000/50000 [==============================] - 58s 1ms/step - loss: 0.3377 - acc: 0.8859
Epoch 5/10
50000/50000 [==============================] - 58s 1ms/step - loss: 0.2057 - acc: 0.9344

(4) batch_size=50   →降低batch_size,時間會拉長,精確度也較差
Epoch 1/5
50000/50000 [==============================] - 67s 1ms/step - loss: 1.2722 - acc: 0.5594
Epoch 2/5
50000/50000 [==============================] - 66s 1ms/step - loss: 0.8335 - acc: 0.7060
Epoch 3/5
50000/50000 [==============================] - 68s 1ms/step - loss: 0.6301 - acc: 0.7811
Epoch 4/5
50000/50000 [==============================] - 66s 1ms/step - loss: 0.4595 - acc: 0.8404
Epoch 5/5
50000/50000 [==============================] - 67s 1ms/step - loss: 0.3202 - acc: 0.8895

(5) 在第2捲積後增加maxpooling   → 時間接近.但準確度上升慢
Epoch 1/5
50000/50000 [==============================] - 61s 1ms/step - loss: 1.3199 - acc: 0.5345
Epoch 2/5
50000/50000 [==============================] - 60s 1ms/step - loss: 0.9111 - acc: 0.6803
Epoch 3/5
50000/50000 [==============================] - 60s 1ms/step - loss: 0.7564 - acc: 0.7336
Epoch 4/5
50000/50000 [==============================] - 60s 1ms/step - loss: 0.6389 - acc: 0.7775
Epoch 5/5
50000/50000 [==============================] - 61s 1ms/step - loss: 0.5400 - acc: 0.8102

(6)將1st之捲積後的MaxPooling2D(pool_size=(3,3)   -> 訓練時間大幅縮短,但精度上升慢
Epoch 1/5
50000/50000 [==============================] - 43s 867us/step - loss: 1.2672 - acc: 0.5561
Epoch 2/5
50000/50000 [==============================] - 41s 827us/step - loss: 0.8642 - acc: 0.6955
Epoch 3/5
50000/50000 [==============================] - 44s 871us/step - loss: 0.6768 - acc: 0.7629
Epoch 4/5
50000/50000 [==============================] - 43s 856us/step - loss: 0.5363 - acc: 0.8132
Epoch 5/5
50000/50000 [==============================] - 41s 823us/step - loss: 0.4181 - acc: 0.8543

(7)用GlobalAveragePooling2D取代Flatten  ->訓練時間小幅縮短,但精度上升更慢
Epoch 1/5
50000/50000 [==============================] - 56s 1ms/step - loss: 1.6386 - acc: 0.3968
Epoch 2/5
50000/50000 [==============================] - 53s 1ms/step - loss: 1.3269 - acc: 0.5193
Epoch 3/5
50000/50000 [==============================] - 53s 1ms/step - loss: 1.2032 - acc: 0.5660
Epoch 4/5
50000/50000 [==============================] - 53s 1ms/step - loss: 1.1311 - acc: 0.5931
Epoch 5/5
50000/50000 [==============================] - 53s 1ms/step - loss: 1.0790 - acc: 0.6115

(8)將第一層maxpooling改到第二層後   →時間大幅增加(比(5)更多),精度接近(5)
Epoch 1/5
50000/50000 [==============================] - 99s 2ms/step - loss: 1.3473 - acc: 0.5259
Epoch 2/5
50000/50000 [==============================] - 99s 2ms/step - loss: 0.9332 - acc: 0.6712
Epoch 3/5
50000/50000 [==============================] - 98s 2ms/step - loss: 0.7767 - acc: 0.7264
Epoch 4/5
50000/50000 [==============================] - 99s 2ms/step - loss: 0.6510 - acc: 0.7720
Epoch 5/5
50000/50000 [==============================] - 99s 2ms/step - loss: 0.5350 - acc: 0.8145


